# Main module
It is written in Jupyter Noteboook for better look

### Preparation

In [1]:
# Import some stuff for plots

%matplotlib inline 
import matplotlib.pyplot as plt

In [2]:
# Import project files

from Generate.Generate import generate_dataset
from Encrypt import Caesar

### Set some constants

In [3]:
file_name = './data/words_list.txt'  # Path to file with list of words

### Data preparation

In [4]:
words = generate_dataset(file_name)

Time elapsed: 277972 msec (0 sec)


In [5]:
print("Set 'words' contains {} elements".format(len(words)))

Set 'words' contains 420394 elements


In [6]:
caesar_3 = Caesar.encrypt(words, 3)
caesar_4 = Caesar.encrypt(words, 4)

Time elapsed: 809972 msec (0 sec)
Time elapsed: 824585 msec (0 sec)
